In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24118
2,Huelva,Confirmados PDIA 14 días,1877
3,Huelva,Tasa PDIA 14 días,"365,7657306545589"
4,Huelva,Confirmados PDIA 7 días,565
5,Huelva,Total Confirmados,24348
6,Huelva,Curados,13432
7,Huelva,Fallecidos,256


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24118.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6424.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 773 personas en los últimos 7 días 

Un positivo PCR cada 291 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24118.0,565.0,1877.0,513170.0,110.099967,365.765731,101.0
Huelva-Costa,14697.0,379.0,1195.0,289548.0,130.893669,412.712227,72.0
Condado-Campiña,6674.0,147.0,523.0,156231.0,94.091442,334.760707,33.0
Huelva (capital),6424.0,186.0,494.0,143837.0,129.313042,343.444315,27.0
Almonte,804.0,41.0,97.0,24507.0,167.299139,395.805280,15.0
Isla Cristina,1974.0,41.0,150.0,21393.0,191.651475,701.163932,10.0
Lepe,1953.0,62.0,178.0,27880.0,222.381636,638.450502,9.0
Cartaya,886.0,26.0,75.0,20083.0,129.462730,373.450182,9.0
Bollullos Par del Condado,577.0,23.0,71.0,14387.0,159.866546,493.501077,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,163.0,5.0,37.0,2885.0,173.310225,1282.495667,0.0
Alájar,16.0,1.0,7.0,759.0,131.752306,922.266140,0.0
Isla Cristina,1974.0,41.0,150.0,21393.0,191.651475,701.163932,10.0
Lepe,1953.0,62.0,178.0,27880.0,222.381636,638.450502,9.0
Villanueva de los Castillejos,146.0,7.0,17.0,2825.0,247.787611,601.769912,3.0
Villalba del Alcor,437.0,2.0,20.0,3366.0,59.417706,594.177065,0.0
Ayamonte,1175.0,26.0,118.0,21104.0,123.199393,559.135709,6.0
Cumbres de San Bartolomé,5.0,2.0,2.0,376.0,531.914894,531.914894,0.0
Beas,158.0,2.0,22.0,4341.0,46.072334,506.795669,-1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Beas,158.0,2.0,22.0,4341.0,46.072334,506.795669,-1.0,0.090909
Villalba del Alcor,437.0,2.0,20.0,3366.0,59.417706,594.177065,0.0,0.100000
Bonares,179.0,2.0,19.0,6060.0,33.003300,313.531353,1.0,0.105263
Villablanca,163.0,5.0,37.0,2885.0,173.310225,1282.495667,0.0,0.135135
Palma del Condado (La),964.0,7.0,50.0,10801.0,64.808814,462.920100,4.0,0.140000
Alájar,16.0,1.0,7.0,759.0,131.752306,922.266140,0.0,0.142857
San Juan del Puerto,353.0,3.0,19.0,9411.0,31.877590,201.891404,0.0,0.157895
Almonaster la Real,54.0,1.0,6.0,1788.0,55.928412,335.570470,-1.0,0.166667
Niebla,158.0,1.0,5.0,4116.0,24.295432,121.477162,0.0,0.200000
